In [1]:
from __future__ import unicode_literals, print_function, division
from io import open
import glob
import os
import unicodedata
import string
import jieba
import torch.nn.functional as F

In [2]:
path = 'data/poetry'

In [3]:
def load_data(path):
    file = path
    datasets = pd.read_csv(file, header=0)
    cats = datasets.cat
    labels = datasets.label
    reviews = datasets.review
    rows = np.random.randint(0, len(datasets), len(datasets))
    return np.array(cats[rows]), np.array(reviews[rows]), np.array(labels[rows])

In [4]:
f = open(path)
a = f.read().split('\n')

In [5]:
print (a)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [6]:
a[0]

'首春:寒随穷律变，春逐鸟声开。初风飘带柳，晚雪间花梅。碧林青旧竹，绿沼翠新苔。芝田初雁去，绮树巧莺来。'

In [7]:
aa = a[0]

In [8]:
pun = [':', '，', '。', '!']
def cutreviews(texts):
    sent_words = []
    for i in range(len(texts)):
        sent_word = list(jieba.lcut(texts[i]))
#         print (sent_word)
        _sent_words = []
        for x in sent_word:
            if x not in pun:
                _sent_words += x
        
        _sent_word = ''.join(str(x) for x in _sent_words)
        sent_words.append(_sent_word)
    return sent_words  

In [9]:
datasets = cutreviews(a)

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.458 seconds.
Prefix dict has been built succesfully.


In [10]:
print (type(datasets))

<class 'list'>


In [11]:
datasets[0]

'首春寒随穷律变春逐鸟声开初风飘带柳晚雪间花梅碧林青旧竹绿沼翠新苔芝田初雁去绮树巧莺来'

## Create Model

In [12]:
import torch
import torch.nn as nn

In [13]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        
        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)
        self.o2o = nn.Linear(hidden_size + output_size, output_size)
        self.dropout = nn.Dropout(0.1)
        self.softmax = nn.LogSoftmax(dim=1)
        
    def forward(self, input, hidden):
        input_combined = torch.cat((input, hidden), 1)
        hidden = self.i2h(input_combined)
        output = self.i2o(input_combined)
        output_combined = torch.cat((hidden, output),1)
        output = self.o2o(output_combined)
        output = self.dropout(output)
#         output = self.softmax(output)
        output = F.log_softmax(output, dim=1)
#         output = output.add(1e-8)
#         output = output.log()
        return output, hidden
    
    def initHidden(self):
        return torch.zeros(1, self.hidden_size)

## Preparing for training

In [14]:
import numpy as np

In [15]:
def randomChoice(data):
    if isinstance(data, list):
        data = np.array(data)
    rows = np.random.randint(0, len(data), len(data))
    return data[rows]

In [16]:
def randomchoiceone(data):
    data = np.array(data)
    return data[np.random.randint(0, len(data)-1)]

In [17]:
ran_datasets = randomChoice(datasets)

In [18]:
ran_datasets[1]

'南亭夜坐贻开元禅定二道者暮暮焚香何处宿西岩一室映疏藤光阴难驻迹如客寒暑不惊心似僧高树有风闻夜磬远山无月见秋灯身闲境静日为乐若问其余非我能'

In [19]:
ran_dataset = randomchoiceone(datasets)

In [20]:
ran_dataset

'宿宣义池亭暮色绕柯亭南山幽竹青夜深斜舫月风定一池星岛屿无人迹菰蒲有鹤翎此中足吟眺何用泛沧溟'

In [21]:
datatest = ran_datasets[:2]

In [22]:
datatest[0]

'清夜忆仙宫子夜静门深紫洞烟孤行独坐忆神仙三清宫里月如昼十二宫楼何处眠'

## Make a vocabulary

In [23]:
def vocabb(data: np.ndarray):
    vocabdict = []
    for i in range(len(data)):
        row = data[i]
        for word in data[i]:
            if word not in vocabdict:
                vocabdict.append(word)
    num = range(0, len(vocabdict)-1)
    
    return dict(zip(vocabdict, num))

In [24]:
vocabl = vocabb(ran_datasets)

In [25]:
def vocabre(vocab: dict):
    return dict((v,k) for k, v in vocab.items())

In [26]:
print (len(vocabl))

7122


In [27]:
vocabl["和"]

122

In [28]:
re_vocab = vocabre(vocabl)

In [29]:
re_vocab[283]

'前'

## One-hot vector for category

In [30]:
def chartoindex(char, vocab):
    num = vocab[char]
    return num

In [31]:
def chartotensor(char, vocab):
    tensor = torch.zeros(1, len(vocab))
    tensor[0][chartoindex(char, vocab)] = 1
    return tensor

In [32]:
def inputtensor(line, vocab):
    tensor = torch.zeros(len(line), 1, len(vocab))
    for li in range(len(line)):
        char = line[li]
        tensor[li][0][vocab[char]] = 1
        return tensor

In [33]:
def targettensor(line, vocab):
    char_indexes = [vocab[line[li]] for li in range(1, len(line))]
    char_indexes.append(len(line))
    return torch.LongTensor(char_indexes)

In [34]:
def randomtraingingexample(data, vocab):
    line = randomchoiceone(data)
    input_line_tensor = inputtensor(line, vocab)
    target_line_tensor = targettensor(line, vocab)
    return input_line_tensor, target_line_tensor

In [35]:
randomtraingingexample(datasets, vocabl)

(tensor([[[0., 0., 0.,  ..., 0., 0., 0.]],
 
         [[0., 0., 0.,  ..., 0., 0., 0.]],
 
         [[0., 0., 0.,  ..., 0., 0., 0.]],
 
         ...,
 
         [[0., 0., 0.,  ..., 0., 0., 0.]],
 
         [[0., 0., 0.,  ..., 0., 0., 0.]],
 
         [[0., 0., 0.,  ..., 0., 0., 0.]]]),
 tensor([ 753, 1698,  594,   68,  417,  589,  867,  492, 1718,  591,  626, 1528,
         1074,   25,  951,  951,  417,  589,  841,   69,   77, 2196,  134,  911,
          535,  133,  371,  315,  320,  213,  301,   43,  149,  810, 1012,  127,
          285,  208,   35,  214, 1087, 1596, 2848,  320,   32, 1501,   47]))

In [36]:
print (inputtensor(datatest[1], vocabl).shape)

torch.Size([68, 1, 7122])


## Train

In [37]:
rnn = RNN(len(vocabl), 1000, len(vocabl)).cuda()
# rnn = PoetryModel(vocabl, 256, 500)
rnn = rnn.cuda()
criterion = nn.NLLLoss()
# criterion = nn.CrossEntropyLoss()
learning_rate = 0.001

optimizer = torch.optim.Adam(rnn.parameters(), learning_rate)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1000, gamma=0.1)

In [38]:
def train(input_line_tensor, target_line_tensor):
    target_line_tensor.unsqueeze_(-1)
    hidden = rnn.initHidden()
    hidden = hidden.cuda()
    #rnn.zero_grad()
    loss = 0
    for i in range(input_line_tensor.size(0)):
        output, hidden = rnn(input_line_tensor[i], hidden)
        optimizer.zero_grad()
        l = criterion(output, target_line_tensor[i])
        loss += l
        optimizer.step()
    loss.backward()
    
    for p in rnn.parameters():
        p.data.add_(-learning_rate, p.grad.data)
        
    return output, loss.item() / input_line_tensor.size(0)

In [39]:
n_iters = 10000
print_every = 200
plot_every = 500
all_losses = []
total_loss = 0

for iter in range(0, n_iters+1):
    input_line_tensor, target_line_tensor = randomtraingingexample(ran_datasets, vocabl)
    input_line_tensor = input_line_tensor.cuda()
    target_line_tensor = target_line_tensor.cuda()
#     print (input_line_tensor)
#     print (target_line_tensor)
    output, loss = train(input_line_tensor, target_line_tensor)
    total_loss += loss
    
    if iter % print_every == 0:
        print('(%d %d%%) %.4f' % (iter, iter / n_iters * 100, loss))
        
    if iter % plot_every == 0:
        all_losses.append(total_loss / plot_every)
        total_loss = 0

(0 0%) 8.8713
(200 2%) 8.8202
(400 4%) 8.7493
(600 6%) nan


KeyboardInterrupt: 

In [ ]:
max_lenght = 7

In [ ]:
def sample(vocab, revocab, start_char='成' ):
    with torch.no_grad():
        input = inputtensor(start_char, vocab)
        input = input.cuda()
        hidden = rnn.initHidden().cuda()
        output_p = start_char
        
        for i in range(max_lenght):
            output, hidden = rnn(input[0], hidden)
            topv, topi = output.topk(1)
            topi = int(topi[0][0])
#            print (topi)
            if topi == len(vocab):
                break
            else:
                char = revocab[topi]
                output_p += char
    return output_p 

In [ ]:
def samples(vocab,revocab, start_chars='天上'):
    for start_char in start_chars:
        print (sample(vocab, revocab, start_char))

In [ ]:
samples(vocabl,re_vocab, start_chars='天上人间')